<a href="https://colab.research.google.com/github/abinash00098/LEARNFLOW/blob/main/E_Commerce_recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score


In [8]:
data = pd.read_csv('1429_1.csv')  # Adjust the file name if needed

# Display the first few rows and data info
print(data.head())
print(data.info())


                     id                                               name  \
0  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   
1  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   
2  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   
3  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   
4  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   

        asins   brand                                         categories  \
0  B01AHB9CN2  Amazon  Electronics,iPad & Tablets,All Tablets,Fire Ta...   
1  B01AHB9CN2  Amazon  Electronics,iPad & Tablets,All Tablets,Fire Ta...   
2  B01AHB9CN2  Amazon  Electronics,iPad & Tablets,All Tablets,Fire Ta...   
3  B01AHB9CN2  Amazon  Electronics,iPad & Tablets,All Tablets,Fire Ta...   
4  B01AHB9CN2  Amazon  Electronics,iPad & Tablets,All Tablets,Fire Ta...   

                                                keys manufacturer  \
0  84

<ipython-input-8-55ef278def33>:1: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('1429_1.csv')  # Adjust the file name if needed


In [18]:
class ElectronicsRecommender:
    def __init__(self, data):
        self.data = data
        self.user_item_matrix = None
        self.user_similarity_matrix = None

    def preprocess_data(self):
        # Create user-item matrix
        self.user_item_matrix = self.data.pivot_table(
            index='reviews.username',
            columns='name',
            values='reviews.rating',
            fill_value=0
        )

    def compute_user_similarity(self):
        # Compute cosine similarity between users
        self.user_similarity_matrix = cosine_similarity(self.user_item_matrix)

    def get_similar_users(self, user, n=5):
        # Get top N similar users
        user_index = self.user_item_matrix.index.get_loc(user)
        similar_users = self.user_similarity_matrix[user_index].argsort()[::-1][1:n+1]
        return self.user_item_matrix.index[similar_users]

    def recommend_products(self, user, n=5):
        similar_users = self.get_similar_users(user)
        user_products = set(self.user_item_matrix.loc[user].index[self.user_item_matrix.loc[user] > 0])

        recommendations = []
        for similar_user in similar_users:
            similar_user_products = set(self.user_item_matrix.loc[similar_user].index[self.user_item_matrix.loc[similar_user] > 0])
            new_products = similar_user_products - user_products
            recommendations.extend(list(new_products))

        # Get top N unique recommendations
        return list(dict.fromkeys(recommendations))[:n]

    def evaluate(self, test_data, k=5):
        precisions = []
        recalls = []
        for user in test_data['reviews.username'].unique():
            if user in self.user_item_matrix.index:
                actual_items = set(test_data[test_data['reviews.username'] == user]['name'])
                recommended_items = set(self.recommend_products(user, n=k))

                tp = len(actual_items.intersection(recommended_items))
                precision = tp / len(recommended_items) if recommended_items else 0
                recall = tp / len(actual_items) if actual_items else 0

                precisions.append(precision)
                recalls.append(recall)

        return np.mean(precisions), np.mean(recalls)

In [19]:
data = pd.read_csv('1429_1.csv')

# Basic data analysis and visualization
plt.figure(figsize=(12, 6))
sns.histplot(data['reviews.rating'], bins=5, kde=True)
plt.title('Distribution of Ratings')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.savefig('rating_distribution.png')
plt.close()

plt.figure(figsize=(12, 6))
top_products = data['name'].value_counts().head(10)
sns.barplot(x=top_products.values, y=top_products.index)
plt.title('Top 10 Most Reviewed Products')
plt.xlabel('Number of Reviews')
plt.savefig('top_products.png')
plt.close()

plt.figure(figsize=(12, 6))
sns.boxplot(x='reviews.rating', y='brand', data=data.sort_values('brand', ascending=False))
plt.title('Distribution of Ratings by Brand')
plt.savefig('brand_ratings.png')
plt.close()


<ipython-input-19-ff621afea063>:1: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('1429_1.csv')
) missing from current font.
  plt.savefig('top_products.png')


In [20]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

recommender = ElectronicsRecommender(train_data)
recommender.preprocess_data()
recommender.compute_user_similarity()


In [21]:
precisions = []
recalls = []
k_values = range(1, 11)

for k in k_values:
    precision, recall = recommender.evaluate(test_data, k=k)
    precisions.append(precision)
    recalls.append(recall)

plt.figure(figsize=(10, 6))
plt.plot(k_values, precisions, marker='o', label='Precision')
plt.plot(k_values, recalls, marker='o', label='Recall')
plt.title('Precision and Recall at K')
plt.xlabel('K')
plt.ylabel('Score')
plt.legend()
plt.savefig('precision_recall_at_k.png')
plt.close()

In [22]:
user = "Adapter"  # Replace with an actual username from your dataset
recommendations = recommender.recommend_products(user)
print(f"Recommendations for {user}:")
for i, product in enumerate(recommendations, 1):
    print(f"{i}. {product}")

# User similarity heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(recommender.user_similarity_matrix[:50, :50], cmap='YlOrRd')
plt.title('User Similarity Heatmap (Top 50 Users)')
plt.savefig('user_similarity_heatmap.png')
plt.close()

print("Analysis complete. Check the generated PNG files for visualizations.")

Recommendations for Adapter:
Analysis complete. Check the generated PNG files for visualizations.
